In [ ]:
from rich import print

from cali._plate_viewer._analysis_with_sqlmodel import AnalysisRunner
from cali.readers import TensorstoreZarrReader
from cali.sqlmodel import print_experiment_tree, save_experiment_to_database

# Run from Database Example

In [ ]:
from cali.sqlmodel import load_experiment_from_database

In [ ]:
database_path = (
    "/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis/cali.db"
)
exp = load_experiment_from_database(database_path)
print_experiment_tree(exp)

In [ ]:
# delete existing rois
for well in exp.plate.wells:
    for fov in well.fovs:
        fov.rois = []
# save database after deletion
save_experiment_to_database(exp, database_path, overwrite=True)

print_experiment_tree(exp)

In [ ]:
runner = AnalysisRunner()
runner.set_experiment(exp)
data = TensorstoreZarrReader(exp.data_path)
# print("data:", data.store)
runner.set_data(data)
# print(runner.get_settings())

In [ ]:
runner.run()

In [ ]:
print_experiment_tree(exp)

# Run From Directories Example

In [ ]:
from datetime import datetime

import useq

from cali.sqlmodel import AnalysisSettings, Experiment, useq_plate_plan_to_db

In [ ]:
database_path = "/Users/fdrgsp/Desktop/cali_test/cali_new.db"
data_path = (
    "/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk.tensorstore.zarr"
)
analysis_path = "/Users/fdrgsp/Desktop/cali_test"
labels_path = "/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_labels"

new_exp = Experiment(
    id=0,
    name="New Experiment",
    description="A Test Experiment.",
    created_at=datetime.now(),
    database_path=database_path,
    data_path=data_path,
    labels_path=labels_path,
    analysis_path=analysis_path,
)
print(new_exp)

In [ ]:
# load the data and get the useq plate plan from the sequence
data = TensorstoreZarrReader(new_exp.data_path)
plate_plan = data.sequence.stage_positions
assert isinstance(plate_plan, useq.WellPlatePlan)

# Define plate maps for conditions
plate_maps = {
    "genotype": {"B5": "WT"},
    "treatment": {"B5": "Vehicle"},
}

# Create plate with plate_maps and conditions in one step
plate = useq_plate_plan_to_db(plate_plan, new_exp, plate_maps)
new_exp.plate = plate

print(new_exp.plate)
print_experiment_tree(new_exp)

In [ ]:
analysis_settings = AnalysisSettings(
    neuropil_inner_radius=0,
    peaks_prominence_multiplier=1.0,
    spikes_sync_cross_corr_lag=5,
    neuropil_min_pixels=0,
    calcium_sync_jitter_window=5,
    led_power_equation="y = 11 * x - 6",
    neuropil_correction_factor=0.0,
    calcium_network_threshold=90.0,
    led_pulse_duration=100,
    led_pulse_powers=[2.0, 4.0, 6.0],
    led_pulse_on_frames=[3, 53, 103],
    decay_constant=0.0,
    spike_threshold_value=1.0,
    stimulation_mask_path="/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis/stimulation_mask.tif",
    id=1,
    dff_window=30,
    spike_threshold_mode="multiplier",
    threads=1,
    peaks_height_value=3.0,
    burst_threshold=30.0,
    experiment_id=1,
    peaks_height_mode="multiplier",
    burst_min_duration=3,
    stimulation_mask_id=1,
    created_at=datetime(2025, 11, 8, 20, 25, 1, 911683),
    peaks_distance=2,
    burst_gaussian_sigma=2.0,
)
new_exp.analysis_settings = analysis_settings

print_experiment_tree(new_exp)

In [ ]:
runner = AnalysisRunner()
runner.set_experiment(new_exp)
data = TensorstoreZarrReader(new_exp.data_path)
runner.set_data(data)

# set which positions to analyze
new_exp.positions_analyzed = list(range(len(plate_plan)))
print(f"Positions to analyze: {new_exp.positions_analyzed}")


def _p(msg: str) -> None:
    print("ANALYSIS INFO:", msg)


runner.analysisInfo.connect(_p)

In [ ]:
runner.run()

In [ ]:
exp_after_analysis = load_experiment_from_database(database_path)
print_experiment_tree(exp_after_analysis)